He ejecutado esto tanto en el servidor remoto con SOCKs como en la máquin WSL local.

In [ ]:
!pip install googletrans==4.0.0-rc1 PySocks

# Googletrans API google translator gratis (usar con cuidado) (usar con proxy socks)

In [ ]:
%pip install googletrans==3.1.0a0

## Sin proxy SOCKs


In [5]:
import socks
import socket
import time
import random
from googletrans import Translator

# IP = "74.208.163.90"
# # Configura el proxy SOCKS
# socks.set_default_proxy(socks.SOCKS5, IP, 5559, True)  # Reemplaza "IP" con tu dirección IP real
# socket.socket = socks.socksocket
# socket.AF_INET = 2  # Establece explícitamente el tipo de socket a IPv4


def traducir_eng_spa(texto):
    traductor = Translator()
    return traductor.translate(texto, src='en', dest='es').text

# Lista de textos a traducir
textos = ["Hello, world!", "How are you?", "This is a test."]

# Iterar sobre los textos y traducir cada uno
for texto in textos:
    texto_traducido = traducir_eng_spa(texto)
    print(texto_traducido)

    # Genera una pausa aleatoria entre 1 y 2 segundos
    time.sleep(random.uniform(1, 2))


¡Hola Mundo!
¿Cómo estás?
Esto es una prueba.


## Usar con proxy socks

In [2]:
puertosocks = 5559

Ejecutar sino existe el fichero ssh_tunnel:

In [90]:
%%writefile ssh_tunnel.sh
#!/bin/bash
ssh -i .ssh/tunel -N root@74.208.163.90 -D "$1"

Overwriting ssh_tunnel.sh


In [89]:
!chmod +x ssh_tunnel.sh

## Creamos un proceso robusto en segundo plano que se encarga de mantener el túnel ssh abierto

In [92]:
import subprocess

# Comando para ejecutar el script con pm2
cmd = ["pm2", "start", "ssh_tunnel.sh", "--", puertosocks]

# Ejecutar el comando
subprocess.run(cmd)


[PM2] Starting /mnt/d/DeveloperIA/video-translator/ssh_tunnel.sh in fork_mode (1 instance)
[PM2] Done.
┌────┬──────────────────┬─────────────┬─────────┬─────────┬──────────┬────────┬──────┬───────────┬──────────┬──────────┬──────────┬──────────┐
│ id │ name             │ namespace   │ version │ mode    │ pid      │ uptime │ ↺    │ status    │ cpu      │ mem      │ user     │ watching │
├────┼──────────────────┼─────────────┼─────────┼─────────┼──────────┼────────┼──────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0  │ neo4j console    │ default     │ N/A     │ fork    │ 27838    │ 3D     │ 0    │ online    │ 0%       │ 7.5mb    │ javier   │ disabled │
│ 5  │ ssh_tunnel       │ default     │ N/A     │ fork    │ 26478    │ 0s     │ 0    │ online    │ 0%       │ 3.2mb    │ javier   │ disabled │
└────┴──────────────────┴─────────────┴─────────┴─────────┴──────────┴────────┴──────┴───────────┴──────────┴──────────┴──────────┴──────────┘


CompletedProcess(args=['pm2', 'start', 'ssh_tunnel.sh', '--', '5559'], returncode=0)

No funciona por culpa de que la contraseña está encriptada en el fichero de configuración de proxy socks.

In [4]:
!pm2 list

┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
│ 5  │ ssh_tunnel         │ fork     │ 0    │ online    │ 0%       │ 3.2mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [85]:
# si queremos borrar el proceso
# !pm2 delete ssh_tunnel

[PM2] Applying action deleteProcessId on app [ssh_tunnel](ids: [ 4 ])
[PM2] [ssh_tunnel](4) ✓
┌────┬────────────────────┬──────────┬──────┬───────────┬──────────┬──────────┐
│ id │ name               │ mode     │ ↺    │ status    │ cpu      │ memory   │
├────┼────────────────────┼──────────┼──────┼───────────┼──────────┼──────────┤
│ 0  │ neo4j console      │ fork     │ 0    │ online    │ 0%       │ 7.5mb    │
└────┴────────────────────┴──────────┴──────┴───────────┴──────────┴──────────┘


In [8]:
!lsof -i :{puertosocks}


COMMAND   PID   USER   FD   TYPE   DEVICE SIZE/OFF NODE NAME
ssh     26479 javier    4u  IPv4 14616200      0t0  TCP localhost:5559 (LISTEN)


In [76]:
# !kill -9 19841

In [95]:
!ps aux | grep 'ssh'

root       240  0.0  0.0  15428  1656 ?        Ss   Nov13   0:00 sshd: /usr/sbin/sshd [listener] 0 of 10-100 startups
javier   14789  0.0  0.0      0     0 ?        Z    16:34   0:00 [sshpass] <defunct>
root     24996  0.0  0.0  16880  4760 ?        Ss   09:45   0:00 sshd: javier [priv]
javier   25022  0.0  0.0  17736  4228 ?        S    09:45   0:01 sshd: javier@notty
javier   26478  0.0  0.0   4780  3324 ?        Ss   17:29   0:00 bash /mnt/d/DeveloperIA/video-translator/ssh_tunnel.sh 5559
javier   26479  0.0  0.0  11716  8192 ?        S    17:29   0:00 ssh -i .ssh/tunel -N root@74.208.163.90 -D 5559
javier   26594  0.0  0.0   4780  3296 pts/0    Ss+  17:29   0:00 /bin/bash -c ps aux | grep 'ssh'
javier   26596  0.0  0.0   4024  2000 pts/0    S+   17:29   0:00 grep ssh
root     28723  0.0  0.0  16884  4688 ?        Ss   10:00   0:00 sshd: javier [priv]
javier   28749  0.0  0.0  18480  5592 ?        S    10:00   0:03 sshd: javier@notty


Ejecutamos servicios mediante el proxy socks

In [6]:
import paramiko
from paramiko import SSHClient
import socket
import socks
import requests
import time
import random
import getpass
from googletrans import Translator


# Configuración del servidor SSH y credenciales
ssh_host = "74.208.163.90"  # IP del servidor SSH
ssh_port = 22               # Puerto SSH
username = "root"     # Tu usuario SSH
path_to_private_key = "./.ssh/tunel"  # Ruta a tu clave privada SSH
# private_key_password = "Cony620438292"#getpass.getpass("Ingresa la frase de contraseña de tu clave privada: ")

try:
    # Configura un cliente SSH y conecta al servidor
    ssh = SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    my_key = paramiko.RSAKey.from_private_key_file(path_to_private_key)
    print(f"Conectando a SSH a {username}@{ssh_host} -p {ssh_port}...")
    try:
        ssh.connect(hostname=ssh_host, port=ssh_port, username=username, pkey=my_key)
        print("Conexión SSH establecida.")
    except paramiko.ssh_exception.AuthenticationException:
        print("Error: credenciales inválidas.")
        exit()
    except Exception as e:
        print(f"Error al principio de establecer la conexión SSH: {e}")
        exit()    

    # Configurar el reenvío de puertos
    local_bind_port = puertosocks
    ssh_transport = ssh.get_transport()
    remote_bind_address = ('localhost', local_bind_port)
    ssh_transport.request_port_forward(address=remote_bind_address[0], port=remote_bind_address[1])

    # Configura el proxy SOCKS
    print(f"Configurando el proxy SOCKS en {remote_bind_address[0]}:{remote_bind_address[1]}...")
    socks.set_default_proxy(socks.SOCKS5, "localhost", local_bind_port, True)
    socket.socket = socks.socksocket
    socket.AF_INET = 2  # IPv4

    def obtener_clima_ny():
        URL = "http://wttr.in/New%20York?format=3"
        print("Realizando solicitud HTTP...")
        respuesta = requests.get(URL, timeout=10)
        return respuesta.text

    # Ejecutar y mostrar la información del clima
    # for _ in range(3):
    try:
        clima = obtener_clima_ny()
        print(clima)
    except Exception as e:
        print(f"Error al realizar la solicitud HTTP: {e}")

        # Pausa
        # time.sleep(random.uniform(1, 2))

    def traducir_eng_spa(texto):
        traductor = Translator()
        return traductor.translate(texto, src='en', dest='es').text

    # Lista de textos a traducir
    textos = ["Hello, world!", "How are you?", "This is a test."]

    # Iterar sobre los textos y traducir cada uno
    for texto in textos:
        try:
            texto_traducido = traducir_eng_spa(texto)
            print(texto_traducido)
        except Exception as e:
            print(f"Error al realizar la traducción: {e}")

        # Pausa
        time.sleep(random.uniform(1, 2))




except Exception as e:
    print(f"Error al establecer la conexión SSH: {e}")

finally:
    # Cerrar la conexión SSH si está abierta
    if ssh:
        ssh.close()
        print("Conexión SSH cerrada.")


Conectando a SSH a root@74.208.163.90 -p 22...
Conexión SSH establecida.
Configurando el proxy SOCKS en localhost:5559...
Realizando solicitud HTTP...
New York: ☀️   +12°C

¡Hola Mundo!
¿Cómo estás?
Esto es una prueba.
Conexión SSH cerrada.


No es necesario hacer esto:
```bash
sudo vi /etc/sysctl.conf
#poner estas lineas al final para desactivar ipv6
net.ipv6.conf.all.disable_ipv6 = 1
net.ipv6.conf.default.disable_ipv6 = 1
net.ipv6.conf.lo.disable_ipv6 = 1


# Para desactivar ipv6 en el kernel
sudo sysctl -p

# Para borrar caches de DNS (depende del sistema, si no se actualiza el DNS)
sudo systemd-resolve --flush-caches

# Para comprobar que se ha desactivado
ip a | grep inet6

# también se puede comprobar con
cat /proc/sys/net/ipv6/conf/all/disable_ipv6
```

In [3]:
# import paramiko
# from paramiko import SSHClient
import socket
import socks
import requests
import time
import random
import getpass
from googletrans import Translator


# Configuración del servidor SSH y credenciales
ssh_host = "74.208.163.90"  # IP del servidor SSH
ssh_port = 22               # Puerto SSH
username = "root"     # Tu usuario SSH
path_to_private_key = "/home/javier/.ssh/id_rsa"  # Ruta a tu clave privada SSH
private_key_password = "Cony620438292"#getpass.getpass("Ingresa la frase de contraseña de tu clave privada: ")

try:
    # # Configura un cliente SSH y conecta al servidor
    # ssh = SSHClient()
    # ssh.load_system_host_keys()
    # ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # my_key = paramiko.RSAKey.from_private_key_file(path_to_private_key, password=private_key_password)
    # print("Conectando a SSH...")
    # ssh.connect(ssh_host, port=ssh_port, username=username, pkey=my_key)
    # print("Conexión SSH establecida.")

    # # Configurar el reenvío de puertos
    # local_bind_port = 5559
    # ssh_transport = ssh.get_transport()
    # remote_bind_address = ('localhost', local_bind_port)
    # ssh_transport.request_port_forward(address=remote_bind_address[0], port=remote_bind_address[1])

    # # Configura el proxy SOCKS
    # socks.set_default_proxy(socks.SOCKS5, "localhost", local_bind_port, True)
    # socket.socket = socks.socksocket
    # socket.AF_INET = 2  # IPv4

    def obtener_clima_ny():
        URL = "http://wttr.in/New%20York?format=3"
        print("Realizando solicitud HTTP...")
        respuesta = requests.get(URL, timeout=10)
        return respuesta.text

    # Ejecutar y mostrar la información del clima
    # for _ in range(3):
    try:
        clima = obtener_clima_ny()
        print(clima)
    except Exception as e:
        print(f"Error al realizar la solicitud HTTP: {e}")

        # Pausa
        # time.sleep(random.uniform(1, 2))

    # def traducir_eng_spa(texto):
    #     traductor = Translator()
    #     return traductor.translate(texto, src='en', dest='es').text

    # # Lista de textos a traducir
    # textos = ["Hello, world!", "How are you?", "This is a test."]

    # # Iterar sobre los textos y traducir cada uno
    # for texto in textos:
    #     try:
    #         texto_traducido = traducir_eng_spa(texto)
    #         print(texto_traducido)
    #     except Exception as e:
    #         print(f"Error al realizar la traducción: {e}")

    #     # Pausa
    #     time.sleep(random.uniform(1, 2))

except Exception as e:
    print(f"Error al establecer la conexión SSH: {e}")

# finally:
    # Cerrar la conexión SSH si está abierta
    # if ssh:
    #     ssh.close()
    #     print("Conexión SSH cerrada.")


Realizando solicitud HTTP...
New York: ☀️   +8°C



In [4]:
from textblob import TextBlob
import time
import random

def traducir_eng_spa(texto):
    # Imprimir el tipo de dato y el contenido de 'texto'
    print(f"Traduciendo: {texto} (tipo: {type(texto)})")

    # Verificar que 'texto' sea una cadena (string)
    if isinstance(texto, str):
        blob = TextBlob(texto)
        return str(blob.translate(to='es'))
    else:
        raise TypeError("El texto a traducir debe ser una cadena (string), no un/a {type(texto)}.")

# Lista de textos a traducir
textos = ["Hello, world!", "How are you?", "This is a test."]

# Iterar sobre los textos y traducir cada uno
for texto in textos:
    try:
        texto_traducido = traducir_eng_spa(texto)
        print(texto_traducido)
    except Exception as e:
        print(f"Error al realizar la traducción: {e}")

    # Pausa
    time.sleep(random.uniform(1, 2))


Traduciendo: Hello, world! (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
Traduciendo: How are you? (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
Traduciendo: This is a test. (tipo: <class 'str'>)
Error al realizar la traducción: 'list' object has no attribute 'strip'
